In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import wget

# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(link).text

### Transform the data in the table on the Wikipedia page into pandas dataframe

In [3]:
soup = BeautifulSoup(source,'lxml')
table = soup.find('table')

In [4]:
column_names = ['Postcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)
n = 1
for data in table.find_all('td'):
    if n%3 == 1:
        Postcode = data.text
    if n%3 == 2:
        Borough = data.text
    if n%3 == 0:
        Neighborhood = data.text
        Neighborhood = Neighborhood.replace('\n','')
        
        df = df.append({'Postcode':Postcode,
                   'Borough':Borough,
                   'Neighborhood':Neighborhood}, ignore_index=True)
    n += 1
    
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore cells with a borough that is Not assigned.

In [5]:
df_cleaned = df[df['Borough']!='Not assigned']
df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Assign neighborhood which is "Not Assigned" to corresponding borough

In [6]:
df_cleaned2 = df_cleaned
df_cleaned2.loc[df_cleaned['Neighborhood']=='Not assigned','Neighborhood'] = df_cleaned.loc[df_cleaned['Neighborhood']=='Not assigned','Borough']
df_cleaned2.head()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Combine rows which has same borough

In [7]:
def joinNbh(list):
    return ', '.join(list)
df_group = df_cleaned2.groupby(['Postcode','Borough'])['Neighborhood'].apply(joinNbh).to_frame().reset_index()
df_group.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df_group.shape

(103, 3)

In [9]:
filename = wget.download('https://cocl.us/Geospatial_data')

100% [................................................................................] 2891 / 2891

In [10]:
df_GC = pd.read_csv(filename)
df_GC.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_group['Latitude'] = df_GC['Latitude']
df_group['Longitude'] = df_GC['Longitude']
df_group.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Generate maps to visualize neighborhoods

#### Use geopy library to get the latitude and longitude values of Toronto.

In [12]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [13]:
df_group['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [14]:
def hasToronto(string):
    if 'Toronto' in string:
        return True
    else:
        return False
    
df_tt = df_group[df_group['Borough'].apply(hasToronto)].reset_index(drop=True)
df_tt= df_tt.drop('Postcode', axis=1)
df_tt.head()

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [15]:
# create map of New York using latitude and longitude values
map_tt = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tt['Latitude'], df_tt['Longitude'], df_tt['Borough'], df_tt['Neighborhood']):
    label = '{}, {}'.format(df_tt, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tt)  
    
map_tt

### Explore the first neighborhood in our dataframe

#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Get the neighborhood's latitude and longitude values.

In [17]:
neighborhood_latitude = df_tt.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_tt.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_tt.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


Get the top 100 venues that are in The Beaches within a radius of 500 meters

In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FM3YM5000OUTOKLWJXNQWS4YR34G4VUOTP3KAANVZDSU3QDC&client_secret=2EHMQKMLH3QRS2Z4BWUDEN4H0OK5M021RODRS3HX1J5X0NPV&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [19]:
results = requests.get(url).json()

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Upper Beaches,Neighborhood,43.680563,-79.292869


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


### Explore Neighborhoods in Manhattan

#### create a function to repeat the same process to all the neighborhoods in Manhattan

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
tt_venues = getNearbyVenues(names=df_tt['Neighborhood'],
                                   latitudes=df_tt['Latitude'],
                                   longitudes=df_tt['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

#### Check the size of the resulting dataframe

In [25]:
print(tt_venues.shape)
tt_venues.head()

(1700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Check how many venues were returned for each neighborhood

In [26]:
tt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


#### Find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(tt_venues['Venue Category'].unique())))

There are 238 uniques categories.


### Analyze Each Neighborhood

In [28]:
# one hot encoding
tt_onehot = pd.get_dummies(tt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tt_onehot['Neighborhood'] = tt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tt_onehot.columns[-1]] + list(tt_onehot.columns[:-1])
tt_onehot = tt_onehot[fixed_columns]

tt_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
tt_onehot.shape

(1700, 238)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
tt_grouped = tt_onehot.groupby('Neighborhood').mean().reset_index()
tt_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.01,0.0,0.0
1,Berczy Park,0.000000,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.00,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.045455,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.0,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0


In [31]:
tt_grouped.shape

(38, 238)

#### Print each neighborhood along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in tt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tt_grouped[tt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4                  Bar  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Italian Restaurant  0.04
3                Café  0.04
4            Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.05
4   Grocery Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.05
1     Auto Workshop  0.05
2              Park  0.05
3        Comic Shop  0.05
4  Recording Studio  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service

#### Put that into a pandas dataframe

Write a function to sort the venues in descending order

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tt_grouped['Neighborhood']

for ind in np.arange(tt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,American Restaurant,Steakhouse,Thai Restaurant,Cosmetics Shop,Hotel,Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Farmers Market,Steakhouse,Bakery,Beer Bar,Italian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Convenience Store,Pet Store,Climbing Gym,Restaurant,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Gym / Fitness Center,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Coffee Shop,Sculpture Garden,Bar,Boutique,Plane,Airport Gate


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [35]:
# set number of clusters
kclusters = 5

tt_grouped_clustering = tt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tt_merged = df_tt

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tt_merged = tt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tt_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Other Great Outdoors,Pub,Trail,Falafel Restaurant,Event Space,Ethiopian Restaurant,Farmers Market,Diner,Fast Food Restaurant
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Pizza Place,Brewery,Bubble Tea Shop,Café
2,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Pizza Place,Italian Restaurant,Pet Store,Gym,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint
3,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym / Fitness Center,Brewery,Seafood Restaurant
4,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


Visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tt_merged['Latitude'], tt_merged['Longitude'], tt_merged['Neighborhood'], tt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [38]:
tt_merged.loc[tt_merged['Cluster Labels'] == 0, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Health Food Store,Other Great Outdoors,Pub,Trail,Falafel Restaurant,Event Space,Ethiopian Restaurant,Farmers Market,Diner,Fast Food Restaurant
1,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Pizza Place,Brewery,Bubble Tea Shop,Café
2,"The Beaches West, India Bazaar",Park,Pizza Place,Italian Restaurant,Pet Store,Gym,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint
3,Studio District,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym / Fitness Center,Brewery,Seafood Restaurant
5,Davisville North,Playground,Hotel,Food & Drink Shop,Clothing Store,Park,Sandwich Place,Breakfast Spot,Grocery Store,Gym,Ethiopian Restaurant
6,North Toronto West,Coffee Shop,Yoga Studio,Bagel Shop,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park,Clothing Store,Chinese Restaurant
7,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Thai Restaurant,Restaurant,Café,Coffee Shop,Italian Restaurant,Deli / Bodega
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,Light Rail Station,Bagel Shop,Fried Chicken Joint,Supermarket,Sports Bar,American Restaurant,Sushi Restaurant,Pizza Place
11,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Café,Italian Restaurant,Bakery,Pub,Pizza Place,Pet Store,Breakfast Spot
12,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Bubble Tea Shop,Burger Joint,Pub,Hotel,Café


#### Cluster 1

In [39]:
tt_merged.loc[tt_merged['Cluster Labels'] == 1, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park, Summerhill East",Playground,Tennis Court,Diner,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 2

In [40]:
tt_merged.loc[tt_merged['Cluster Labels'] == 2, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,Park,Playground,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
23,"Forest Hill North, Forest Hill West",Trail,Jewelry Store,Sushi Restaurant,Park,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Women's Store


#### Cluster 3

In [41]:
tt_merged.loc[tt_merged['Cluster Labels'] == 3, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,Ice Cream Shop,Garden,Women's Store,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 4

In [42]:
tt_merged.loc[tt_merged['Cluster Labels'] == 4, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,Park,Swim School,Bus Line,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
